In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.path as mpath
from matplotlib.gridspec import GridSpec
import datetime
import pandas as pd
from dateutil.parser import parse
from glob import glob
import xarray as xr
import os
import math
from scipy.stats import mode
from scipy.interpolate import griddata, NearestNDInterpolator
from scipy.ndimage.filters import uniform_filter1d
import scipy.ndimage as ndimage


import pickle
from scipy import stats, signal, interpolate

dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')

import warnings
warnings.filterwarnings('ignore')

output_folder = '/pl/active/icesheetsclimate/firn_iceshelves/deg0C/'

In [3]:
#Thickness-permeability relationships
def TP(thickness, A, cut = True):
    if thickness < 0.1: perm = 1
    elif thickness >1:perm = 0
    elif thickness > 0.5 and cut: perm = 0
    else: perm = A**(-1*A*(thickness-0.1))
    return perm

def TP_linear(thickness):
    if thickness < 0.1: perm = 1
    elif thickness>1:perm=0
    else: perm = 1-(1.1*thickness-0.1)
    return perm

In [5]:
def get_pickle_data(site): #read in pickle data files
    site_file = f'{output_folder}pickles/{site}.p'
    with open(site_file, 'rb') as fp:
        data = pickle.load(fp)
    return data

def get_dh(depth): #get thickness of each layer
    dh = np.zeros(depth.shape[0]-1)
    for i in range(len(depth)-1):
        dh[i] = depth[i]-depth[i+1]
    return dh

def get_lenses(item): #get ice lenses in column
    density = np.flip(item['density'])
    depth = list(item['depth'])
    depth.insert(0,0)
    depth = np.flip(np.asarray(depth))
    ice = density > 830
    dh = get_dh(depth)
    dh_lenses = dh*ice

    current_index, current_lens = 0,0
    prev = 0
    lenses=list()
    lens_start = list()
    lens_end = list()

    for i in range(len(dh_lenses)):
        if prev==0 and dh_lenses[i]!=0:
            current_lens = current_lens+dh_lenses[i]
            current_index = i
        elif prev !=0 and dh_lenses[i]!=0:
            current_lens = current_lens+dh_lenses[i]
        elif prev !=0 and dh_lenses[i]==0:
            lenses.append(current_lens)
            lens_start.append(current_index)
            lens_end.append(i)
            current_lens = 0
        prev = dh_lenses[i]

    if current_lens > 0:
        lenses.append(current_lens)
        lens_start.append(current_index)
        lens_end.append(i)

    return lenses, lens_start, lens_end
    

def get_FAC_ts(data,n):
    #n = thickness-permeability relationship used

    FAC_ts = list()

    for item in data:

        l, ls, le = get_lenses(item)
        air = np.flip(item['air'])
        depth = list(item['depth'])
        depth.insert(0,0)
        depth = np.flip(np.asarray(depth))
        dh = get_dh(depth)
        air_m = air*dh

        if len(l) > 0:
            first_lens_start = ls[0]
            first_lens_end = le[0]
            FAC = np.sum(air_m[0:first_lens_start])
            #print(FAC)
            for i in range(len(l)-1):
                lens_start = ls[i+1]
                lens_end = le[i]
                lens_thick = l[i]
                if n == 1: perm = TP(lens_thick,10)
                elif n == 2: perm = TP(lens_thick,4, cut = False)
                else: perm = TP_linear(lens_thick)
                if perm==0:
                    break
                else:
                    FAC = FAC + perm*np.sum(air_m[le[i]:ls[i+1]])
        else: FAC = np.sum(air_m)

        FAC_ts.append(FAC)
        
    return FAC_ts

def get_monthly_fac(data,n): #timeseries of monthly FAC
    FAC_site = get_FAC_ts(data[scenario]['pro'],n)
    FAC_dates = list()
    for item in data[scenario]['pro']:
        FAC_dates.append(item['date'])

    fac_data = {'time':FAC_dates, 'FAC': FAC_site}
    df = pd.DataFrame(fac_data)
    year_month_idx = pd.MultiIndex.from_arrays([df.time.dt.year, df.time.dt.month])
    df['year_month'] = year_month_idx
    fac_monthly = df.groupby('year_month').mean().values
    
    return fac_monthly

def get_monthly_climate(data): #timeseries of monthly climate data
    smet = data[scenario]['ts']
    smet.time = smet.time - pd.DateOffset(hours=6)
    if scenario == 'hist':
        smet = smet[smet.time.dt.year >=1985]
    smet = smet.set_index('time')
    monthly = smet.groupby(pd.Grouper(freq="M"))
    monthly_sum = monthly.sum()
    monthly_mean = monthly.mean()


    rain = monthly_sum.rainfall.values
    snow = monthly_sum.snowfall.values
    melt = monthly_sum['melt'].values
    refreeze = monthly_sum['freeze'].values
    wind = monthly_mean.VW.values
    ta = monthly_mean.TA.values
    
    return rain, snow, melt, refreeze, wind, ta 
    
def save_ds(lats, lons, rain, snow, melt, refreeze, ta, wind, fac, scenario): #save data to netcdf
    if scenario=='hist':time = pd.date_range('1985-01-01', '2014-12-31', freq='1M')
    else: time = pd.date_range('2015-01-01', '2099-12-31', freq='1M')
    ds = xr.Dataset(
        data_vars=dict(
            lat=(["site"], lats),
            lon=(["site"], lons),
            rain=(["site", "time"], rain),
            snow=(["site", "time"], snow),
            melt=(["site", "time"], melt),
            refreeze=(["site", "time"], refreeze),
            ta=(["site", "time"], ta),
            wind=(["site", "time"], wind),
            fac=(["site", "time"], fac),
        ),
        coords=dict(
            site=sites,
            time=time,
        ),
        attrs=dict(rain = 'kg/m2', snow = 'kg/m2',melt = 'kg/m2',refreeze = 'kg/m2',wind = 'm/s',ta = 'K',fac = 'm'),
    )
    ds.to_netcdf(f'netcdfs/{run}/{scenario}_alldata.nc')

In [6]:
with open('site_coords.p', 'rb') as fp:
    coords = pickle.load(fp)

In [8]:
run = 'icelens_linear'
v = 3 #version: 1: gamma = 10 (TP1), 2: gamma = 4, 3:linear

pickle_files = glob(f'{output_folder}pickles/*.p')
nn=len(pickle_files)

scenarios = ['hist', 'ssp1', 'ssp3', 'ssp5']

for scenario in scenarios:
    print(f'........{scenario}........')
    if scenario == 'hist': t = 360
    else: t = 1020

    rain = np.zeros((nn, t))
    snow = np.zeros((nn, t))
    melt = np.zeros((nn, t))
    refreeze = np.zeros((nn, t))
    wind = np.zeros((nn, t))
    ta = np.zeros((nn, t))
    fac = np.zeros((nn, t))
    lats = np.zeros(nn)
    lons = np.zeros(nn)

    n=1
    sites=list()
    for f in pickle_files: #go through and get climate data and FAC for each site
        site = f.split('/')[-1][:-2]
        print(f'{site}, {n}/{nn}')

        data = get_pickle_data(site)
        fac_site = get_monthly_fac(data,v) 
        rain_site, snow_site, melt_site, refreeze_site, wind_site, ta_site = get_monthly_climate(data)
        lat_site = coords[site]['lat']
        lon_site = coords[site]['lon']

        rain[n-1,:] = rain_site
        snow[n-1,:] = snow_site
        melt[n-1,:] =  melt_site
        refreeze[n-1,:] = refreeze_site
        wind[n-1,:] = wind_site
        ta[n-1,:] = ta_site
        fac[n-1,:] = fac_site.flatten()
        lats[n-1] = lat_site
        lons[n-1] = lon_site
        sites.append(site)
        n=n+1
        
    save_ds(lats, lons, rain, snow, melt, refreeze, ta, wind, fac, scenario) #save data to netcdf file

........hist........
VIR102, 1/168
VIR623, 2/168
VIR516, 3/168
VIR598, 4/168
VIR123, 5/168
VIR305, 6/168
VIR602, 7/168
VIR711, 8/168
VIR118, 9/168
VIR554, 10/168
VIR15, 11/168
VIR582, 12/168
VIR139, 13/168
VIR466, 14/168
VIR575, 15/168
VIR640, 16/168
VIR481, 17/168
VIR592, 18/168
VIR608, 19/168
VIR743, 20/168
VIR565, 21/168
VIR588, 22/168
VIR527, 23/168
VIR334, 24/168
VIR720, 25/168
VIR288, 26/168
VIR507, 27/168
VIR67, 28/168
VIR207, 29/168
VIR670, 30/168
VIR264, 31/168
VIR564, 32/168
VIR5, 33/168
VIR356, 34/168
VIR593, 35/168
VIR619, 36/168
VIR752, 37/168
VIR641, 38/168
VIR583, 39/168
VIR425, 40/168
VIR517, 41/168
VIR327, 42/168
VIR215, 43/168
VIR712, 44/168
VIR365, 45/168
VIR695, 46/168
VIR662, 47/168
VIR581, 48/168
VIR257, 49/168
VIR591, 50/168
VIR172, 51/168
VIR266, 52/168
VIR454, 53/168
VIR92, 54/168
VIR611, 55/168
VIR224, 56/168
VIR723, 57/168
VIR505, 58/168
VIR416, 59/168
VIR722, 60/168
VIR148, 61/168
VIR110, 62/168
VIR525, 63/168
VIR703, 64/168
VIR648, 65/168
VIR64, 66/168
VIR6